In [1]:
from datetime import datetime
import os
import pickle
import neat
import numpy as np
from Neat.evaluation_util import DataManager
from Neat.selection_util import TournamentReproduction, FitnessPropReproduction
from data_load_util import load_state_data, make_dataset
from tqdm import tqdm
import random
from Neat.saving_util import *
from Neat.neat_util import *

Loading data_manager for NEAT
Loading Data
number of zip codes: 10559
number of zip codes with solar data: 10559
number of zip codes with census data: 10559
Removing Outliers
zips after removing outliers: 7795


In [2]:
config_path = 'Neat/neat-config'

#load datasets
combined_df = make_dataset(remove_outliers=True)
state_df = load_state_data(combined_df, load="Clean_Data/data_by_state.csv")
data_manager = DataManager(combined_df, state_df)
k_folds = 5

Loading Data
number of zip codes: 10559
number of zip codes with solar data: 10559
number of zip codes with census data: 10559
Removing Outliers
zips after removing outliers: 7795


In [3]:
#single model run
network = run(config_path, eval_genomes_lexicase)
save_model(network, model_name="NEAT_model1M.pkl")
cv_order = run_network(network, data_manager, cross_val=True)

NUM_PANELS=1000000 NUM_GENERATIONS=3 METRIC_WEIGHTS=[1, 1, 1, 1] OVERALL_THRESHOLD=0.3
loading configuration...
creating population...
setting reporters...
training model...

 ****** Running generation 0 ****** 



100%|██████████| 100/100 [07:39<00:00,  4.60s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 3.09070 - size: (1, 6) - species 1 - id 17
Average adjusted fitness: nan
Mean genetic distance 1.292, standard deviation 0.433
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100      3.1      nan     0
Total extinctions: 0
Generation time: 1902.627 sec
Saving checkpoint to Neat/checkpoints/neat-checkpoint-0

 ****** Running generation 1 ****** 



100%|██████████| 100/100 [10:10<00:00,  6.11s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 3.42654 - size: (2, 7) - species 1 - id 109
Average adjusted fitness: nan
Mean genetic distance 1.511, standard deviation 0.476
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100      3.4      nan     0
Total extinctions: 0
Generation time: 1497.476 sec (1700.052 average)
Saving checkpoint to Neat/checkpoints/neat-checkpoint-1

 ****** Running generation 2 ****** 



100%|██████████| 100/100 [06:10<00:00,  3.71s/it]


Population's average fitness: -inf stdev: nan
Best fitness: 3.40299 - size: (1, 3) - species 1 - id 233
Average adjusted fitness: nan
Mean genetic distance 1.506, standard deviation 0.468
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    2   100      3.4      nan     1
Total extinctions: 0
Generation time: 1204.916 sec (1535.006 average)
Saving checkpoint to Neat/checkpoints/neat-checkpoint-2

Best individual in generation 2 meets fitness threshold - complexity: (2, 7)
directory name: Neat/models\12-25-24
making directory


In [ ]:

#run lexicase
lexi_network, lexi_results = K_fold_run(config_path, data_manager, eval_genomes_lexicase, k=k_folds)
save_model(lexi_network, lexi_results, model_name="NEAT_model_lexicase.pkl", results_name="lexicase_results.pkl")

#run fitness prop
fp_network, fp_results = K_fold_run(config_path, data_manager, eval_genomes_weighted_sum, reproduction_method=FitnessPropReproduction, k=k_folds)
save_model(fp_network, fp_results, model_name="NEAT_model_fitness_prop.pkl", results_name="fitness_prop_results.pkl")

#tournament selection
tourney_network, tourney_results = K_fold_run(config_path, data_manager, eval_genomes_weighted_sum, reproduction_method=TournamentReproduction, k=k_folds)
save_model(tourney_network, model_name="NEAT_model_tournament.pkl", results_name="tournament_results.pkl")

#run random selection
rand_network, rand_results = K_fold_run(config_path, data_manager, eval_genomes_random, k=k_folds)
save_model(rand_network, model_name="NEAT_model_random.pkl")

#print results
result_metrics = ['income_equity', 'racial_equity', 'carbon_offset', 'energy_generation']
for i, res in enumerate(lexi_results):
    print(f"Lexicase {result_metrics[i]}", res)

for i, res in enumerate(fp_results):
    print(f"Fitness prop {result_metrics[i]}", res)
    
for i, res in enumerate(tourney_results):
    print(f"Tourney {result_metrics[i]}", res)

for i, res in enumerate(rand_results):
    print(f"Random {result_metrics[i]}", res)

# for i, res in enumerate(rand_results):
#     print(f"Random {result_metrics[i]}", res)


NameError: name 'K_fold_run' is not defined